# `SELECT`

Nesta aula vamos explorar o comando `SELECT`, usado para consultar a base de dados. 

## Explorando a estrutura da base de dados

vamos a alguns comandos básicos para explorar e entender melhor uma base de dados na primeira consulta

In [1]:
# Configuração inicial do banco de dados
import sqlite3
import os
import sys

# Adicionando o diretório de testes ao path
from tests.tests_select import *

# Conectando ao banco de dados
conn = sqlite3.connect('consultoria.db')

# Habilitando suporte a chaves estrangeiras (muito importante!)
conn.execute("PRAGMA foreign_keys = ON;")

print("✅ Conectado ao banco de dados 'consultoria.db'")
print("✅ Chaves estrangeiras habilitadas")


def validate_and_execute(query, test_function):
    """Valida query em ambiente isolado antes de executar no banco principal"""
    if not query or not query.strip():
        print("❌ Query vazia! Escreva sua query SQL antes de executar.")
        return False
    
    
    try:
        # Testar a query em um banco temporário isolado
        test_conn = sqlite3.connect(':memory:')
        test_conn.execute("PRAGMA foreign_keys = ON;")
        
        # Copiar estrutura existente para o banco de teste
        for linha in conn.iterdump():
            if not linha.startswith('BEGIN') and not linha.startswith('COMMIT'):
                try:
                    test_conn.execute(linha)
                except:
                    pass  # Ignora erros de estrutura já existente
        
        # Testar a query do usuário no banco isolado
        test_conn.execute(query)
        test_conn.commit()
        
        # Se chegou até aqui, a query é válida sintaticamente
        # Agora executar no banco principal
        conn.execute(query)
        conn.commit()
        
        # Testar se o resultado está correto
        success = test_function(conn,query)
        
        if success:
            print("✅ Query executada e validada com sucesso!")
            return True
        else:
            # Se teste falhar, fazer rollback
            print("💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!")
            return False
            
    except Exception as e:
        print(f"❌ Erro na query: {str(e)}")
        print("💡 Corrija a sintaxe e tente novamente!")
        return False
    finally:
        if 'test_conn' in locals():
            test_conn.close()


def execute_query(query):
    """Executa uma query e exibe os resultados de forma formatada"""
    if not query or not query.strip():
        print("❌ Query vazia! Forneça uma query SQL para executar.")
        return
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        # Determinar se é um SELECT ou outro tipo de query
        query_type = query.strip().upper().split()[0]
        
        if query_type == 'SELECT':
            # Para SELECT, buscar e exibir os resultados
            results = cursor.fetchall()
            
            # Obter nomes das colunas
            column_names = [description[0] for description in cursor.description]
            
            if not results:
                print("📄 Nenhum resultado encontrado.")
                return
            
            # Calcular largura das colunas para formatação
            col_widths = []
            for i, col_name in enumerate(column_names):
                max_width = len(col_name)
                for row in results:
                    if row[i] is not None:
                        max_width = max(max_width, len(str(row[i])))
                col_widths.append(max_width + 2)  # +2 para espaçamento
            
            # Imprimir cabeçalho
            print("\n" + "=" * (sum(col_widths) + len(column_names) - 1))
            header = "|".join(col_name.ljust(col_widths[i]) for i, col_name in enumerate(column_names))
            print(header)
            print("-" * (sum(col_widths) + len(column_names) - 1))
            
            # Imprimir dados
            for row in results:
                formatted_row = "|".join(
                    str(value).ljust(col_widths[i]) if value is not None else "NULL".ljust(col_widths[i])
                    for i, value in enumerate(row)
                )
                print(formatted_row)
            
            print("=" * (sum(col_widths) + len(column_names) - 1))
            print(f"📊 Total de registros: {len(results)}")
            
        else:
            # Para INSERT, UPDATE, DELETE, etc.
            conn.commit()
            rows_affected = cursor.rowcount
            
            if query_type in ['INSERT', 'UPDATE', 'DELETE']:
                print(f"✅ {query_type} executado com sucesso!")
                if rows_affected >= 0:
                    print(f"📈 Linhas afetadas: {rows_affected}")
            else:
                print(f"✅ Comando {query_type} executado com sucesso!")
                
    except Exception as e:
        print(f"❌ Erro ao executar query: {str(e)}")
    finally:
        if 'cursor' in locals():
            cursor.close()


def test_and_rollback(test_function):
    """Mantido para compatibilidade - executa teste simples"""
    success = test_function(conn)
    if not success:
        print("💡 Execute a query correta primeiro!")
    return success

print("Sistema configurado")

✅ Conectado ao banco de dados 'consultoria.db'
✅ Chaves estrangeiras habilitadas
Sistema configurado


### O que é um Schema?

O schema é a estrutura que define como os dados são organizados em um banco de dados. É como a "planta baixa" que mostra:

1. **Tabelas**: Quais "gavetas" temos para guardar os dados
   - Ex: `clientes`, `projetos`, `consultores`

2. **Colunas**: Que informações guardamos em cada tabela
   - Ex: em `clientes` temos: id, nome, setor, cidade

3. **Tipos de Dados**: Que tipo de valor cada coluna aceita
   - `INTEGER`: números inteiros (1, 2, 3)
   - `TEXT`: textos ("Ana", "São Paulo")
   - `DATE`: datas (2024-01-15)

4. **Relacionamentos**: Como as tabelas se conectam
   - Ex: cada `projeto` pertence a um `cliente`

### Comandos para Explorar o Schema

Em outros bancos SQL (como MySQL), usamos:
- `SHOW TABLES;` para listar todas as tabelas
- `DESCRIBE tabela;` para ver a estrutura de uma tabela

Porém, no SQLite esses comandos não existem!
 
Mas conseguimos o mesmo resultado usando:

```sql
-- Equivalente ao SHOW TABLES:
SELECT name FROM sqlite_master WHERE type='table';

-- Equivalente ao DESCRIBE tabela:
PRAGMA table_info(nome_da_tabela);
```

### Antes de Começar com SELECT vamos ter uma visao detalhada do nosso Schema:

# Diagrama do Banco de Dados

![Diagrama EER](assets/diagram.png)



In [2]:
execute_query("SELECT * FROM clientes")


id  |nome                 |setor        |cidade               |uf  
-------------------------------------------------------------------
1   |TechCorp Solutions   |Tecnologia   |São Paulo            |SP  
2   |Verde Agro Ltda      |Agronegócio  |Campinas             |SP  
3   |MetalMax Indústrias  |Metalurgia   |Santos               |SP  
4   |EduCare Ensino       |Educação     |Ribeirão Preto       |SP  
5   |FastLogistic S.A.    |Logística    |São José dos Campos  |SP  
📊 Total de registros: 5


## Para selecioanar uma coluna especifica:
```sql
SELECT {coluna} FROM {tabela}
```

In [3]:
#TODO Preencha a para selecionar as colunas setor,cidade e uf:
select_x_query = '''
    SELECT setor, cidade, uf FROM clientes;
'''

validate_and_execute(select_x_query, test_select_scu_clientes)


RESULTADO ESPERADO:
      setor              cidade uf
 Tecnologia           São Paulo SP
Agronegócio            Campinas SP
 Metalurgia              Santos SP
   Educação      Ribeirão Preto SP
  Logística São José dos Campos SP

📊 SEU RESULTADO:
      setor              cidade uf
 Tecnologia           São Paulo SP
Agronegócio            Campinas SP
 Metalurgia              Santos SP
   Educação      Ribeirão Preto SP
  Logística São José dos Campos SP


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

In [4]:
# DESCOMENTE E EXECUTE ESSA CÉLULA APENAS 1 VEZ

execute_query("""
INSERT INTO clientes(nome, setor, cidade, uf)
   VALUES
   ('Mineração Ouro Verde', 'Mineração', 'Belo Horizonte', 'MG'),
   ('Café Premium Montanhas', 'Agronegócio', 'Uberlândia', 'MG'),
   ('PetroSul Distribuidora', 'Petróleo', 'Rio de Janeiro', 'RJ'),
   ('BeachTech Inovação', 'Tecnologia', 'Niterói', 'RJ'),
   ('TurisRio Hospedagem', 'Turismo', 'Cabo Frio', 'RJ')
""")

✅ INSERT executado com sucesso!
📈 Linhas afetadas: 5


### Agora vamos filtrar ainda mais, selecione apenas os clientes do estado de Sao Paulo

```sql
SELECT {} FROM tabela WHERE {coluna} LIKE {regra}
```

In [5]:
execute_query(" SELECT * FROM clientes WHERE uf LIKE 'sp' ")


id  |nome                 |setor        |cidade               |uf  
-------------------------------------------------------------------
1   |TechCorp Solutions   |Tecnologia   |São Paulo            |SP  
2   |Verde Agro Ltda      |Agronegócio  |Campinas             |SP  
3   |MetalMax Indústrias  |Metalurgia   |Santos               |SP  
4   |EduCare Ensino       |Educação     |Ribeirão Preto       |SP  
5   |FastLogistic S.A.    |Logística    |São José dos Campos  |SP  
📊 Total de registros: 5


In [7]:
#TODO agora selecione os clientes que o setor começa com T

select_x2_query = '''
    SELECT * FROM clientes WHERE setor LIKE 'T%';
'''

validate_and_execute(select_x2_query, test_select_2_clientes)


RESULTADO ESPERADO:
 id                nome      setor    cidade uf
  1  TechCorp Solutions Tecnologia São Paulo SP
  9  BeachTech Inovação Tecnologia   Niterói RJ
 10 TurisRio Hospedagem    Turismo Cabo Frio RJ

📊 SEU RESULTADO:
 id                nome      setor    cidade uf
  1  TechCorp Solutions Tecnologia São Paulo SP
  9  BeachTech Inovação Tecnologia   Niterói RJ
 10 TurisRio Hospedagem    Turismo Cabo Frio RJ


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

## Vamos fazer alguns exercicios para praticar:

In [8]:
#TODO selecione as alocacoes com mais de 100 horas trabalhadas 
select_x3_query = '''
    SELECT * FROM alocacoes WHERE HORAS_TRABALHADAS > 100;    
'''

validate_and_execute(select_x3_query, test_select_3)


RESULTADO ESPERADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  1           1             1              120.5
  4           2             5              110.0
  5           3             1              150.0
  7           4             5              200.0

📊 SEU RESULTADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  1           1             1              120.5
  4           2             5              110.0
  5           3             1              150.0
  7           4             5              200.0


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

Tambem podemos usar o ```ORDER BY``` para ordenar as linhas da resposta
e adicioanar um ```DESC``` para influenciar na ordem

In [10]:
#TODO Ordene a resposta acima por quem trabalhou mais horas

select_x4_query = '''
    SELECT * FROM alocacoes WHERE HORAS_TRABALHADAS > 100 ORDER BY HORAS_TRABALHADAS DESC;
'''

validate_and_execute(select_x4_query, test_select_4)


RESULTADO ESPERADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  7           4             5              200.0
  5           3             1              150.0
  1           1             1              120.5
  4           2             5              110.0

📊 SEU RESULTADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  7           4             5              200.0
  5           3             1              150.0
  1           1             1              120.5
  4           2             5              110.0


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

### Mas e se quisermos ver apenas a alocacao com mais horas trabalhadas?
### Em vez de receber uma lista com todas, receber uma única alocacao?
### Para isso utilizamos o ```LIMIT```, que limita a quantidade de linhas retornadas

In [12]:
#TODO Devolva apenas o projeto com mais horas trabalhadas

select_x5_query = '''
    SELECT * FROM alocacoes ORDER BY HORAS_TRABALHADAS DESC LIMIT 1;
'''

validate_and_execute(select_x5_query, test_select_5)


RESULTADO ESPERADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  7           4             5              200.0

📊 SEU RESULTADO:
 id  projeto_id  consultor_id  horas_trabalhadas
  7           4             5              200.0


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

### AS
 Podemos usar o ```as``` quando o nome de uma coluna deve ser retornado de forma diferente do original

```sql
 SELECT {nome_coluna} as {novo_nome}```

In [13]:
execute_query("SELECT titulo as nome_do_projeto FROM projetos")


nome_do_projeto  
-----------------
ERP I            
MARKETING II     
LOGISTICA III    
EDUCACAO IV      
VENDAS V         
FINANCEIRO VI    
📊 Total de registros: 6


### Agora que voce ja tem nocoes basicas de SQL, vamos para alguns exercicios práticos:

## Exercicio ```01```:
### Qual o id do projeto que teve a pior avaliação? Retorne em uma coluna chamada ```pior_projeto```

In [16]:
#TODO Escreva aqui sua query:
select_ex1_query = '''
    SELECT id AS pior_projeto FROM feedbacks ORDER BY nota ASC LIMIT 1;
'''

validate_and_execute(select_ex1_query, test_ex1)


RESULTADO ESPERADO:
 pior_projeto
            5

📊 SEU RESULTADO:
 pior_projeto
            4


❌ FALHOU: Os resultados não são iguais
💡 Dica: Verifique se você selecionou apenas as colunas pedidas na ordem correta
💡 Query executada mas resultado incorreto. Banco restaurado, tente novamente!


False

## Exercicio ```02```
### Quais projetos ainda nao terminaram? Retorno os nomes do projeto em uma coluna chamada ```nao_acabou```

```DICA:``` NULL

In [17]:
selext_ex2_query = '''
    SELECT titulo AS nao_acabou FROM projetos WHERE data_fim IS NULL;
'''

validate_and_execute(selext_ex2_query, test_ex2)


RESULTADO ESPERADO:
   nao_acabou
LOGISTICA III
     VENDAS V
FINANCEIRO VI

📊 SEU RESULTADO:
   nao_acabou
LOGISTICA III
     VENDAS V
FINANCEIRO VI


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

## Exercicio ```03```
## Quais projetos começaram antes de abril e já foram entregues?
Retorne lista chamada 'finalizados'

```DICA:```AND e MONTH


In [18]:
select_ex3_query = '''
    SELECT titulo AS finalizados FROM projetos WHERE strftime('%m', data_inicio) < '04' AND data_fim IS NOT NULL;
'''

validate_and_execute(select_ex3_query,test_ex3)


RESULTADO ESPERADO:
 finalizados
       ERP I
MARKETING II

📊 SEU RESULTADO:
 finalizados
       ERP I
MARKETING II


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

## Exercicio ```04```
### Agora filtre as alocacoes que trabalharam entre 80 e 100 horas

```DICA:``` BETWEEN

In [20]:
select_ex4_query='''
    SELECT horas_trabalhadas FROM alocacoes WHERE horas_trabalhadas BETWEEN 80 AND 100;
'''

validate_and_execute(select_ex4_query, test_ex4)


RESULTADO ESPERADO:
 horas_trabalhadas
              80.0
              95.5
              85.5
              90.0

📊 SEU RESULTADO:
 horas_trabalhadas
              80.0
              95.5
              85.5
              90.0


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True

## Exercicio ```05```
### Retorne uma lista com o nome dos clientes que moram em SP ou que são do Agronegócio


In [21]:
select_ex5_query = '''
    SELECT nome FROM clientes WHERE uf = 'SP' OR setor = 'Agronegócio';

'''

validate_and_execute(select_ex5_query, test_ex5)


RESULTADO ESPERADO:
                  nome
    TechCorp Solutions
       Verde Agro Ltda
   MetalMax Indústrias
        EduCare Ensino
     FastLogistic S.A.
Café Premium Montanhas

📊 SEU RESULTADO:
                  nome
    TechCorp Solutions
       Verde Agro Ltda
   MetalMax Indústrias
        EduCare Ensino
     FastLogistic S.A.
Café Premium Montanhas


✅ PASSOU: SELECT - setor, cidade e uf dos clientes
✅ Query executada e validada com sucesso!


True